In [1]:
!pip install nhanes
!pip install pyjanitor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 2.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nhanes.load
import sklearn.compose
import sklearn.preprocessing
import janitor

# CARGAMOS NUESTRO DATASET

In [3]:
nhanes_raw_df = (
    nhanes.load.load_NHANES_data(year="2017-2018")
    .clean_names(
        case_type="snake"
    )
)
nhanes_raw_df.head(2)

,general_health_condition,ever_breastfed_or_fed_breastmilk,age_stopped_breastfeedingdays,age_first_fed_formuladays,age_stopped_receiving_formuladays,age_started_other_foodbeverage,age_first_fed_milkdays,type_of_milk_first_fed_whole_milk,type_of_milk_first_fed2_milk,type_of_milk_first_fed1_milk,...,days_smoked_cigs_during_past30_days,avg_cigarettesday_during_past30_days,tried_to_quit_smoking,times_stopped_smoking_cigarettes,how_long_were_you_able_to_stop_smoking,unit_of_measure_dayweekmonthyear_2_smq,current_selfreported_height_inches,current_selfreported_weight_pounds,tried_to_lose_weight_in_past_year,times_lost10_lbs_or_more_to_lose_weight
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,NaN,1.0,273.0,1.0,365.0,152.0,365.0,Whole or regular milk,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,1.0,60.0,3.0,365.0,126.0,365.0,Whole or regular milk,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# CREAMOS UN DATAGRAME MAS PEQUEÑO

In [5]:
nhanes_df =  (
    nhanes_raw_df.select_columns(
        "general_health_condition",
        "age_in_years_at_screening",
        "gender"
    )
).rename_columns(
    {
        "age_in_years_at_screening":"age"
    }
)
nhanes_df

/usr/local/lib/python3.11/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `jn.select` instead.
  return method(self._obj, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


,general_health_condition,age,gender
SEQN,,,
93703.0,NaN,2.0,Female
93704.0,NaN,2.0,Male
93705.0,Good,66.0,Female
93706.0,Very good,18.0,Male
93707.0,Good,13.0,Male
...,...,...,...
102952.0,Very good,70.0,Female
102953.0,Fair or,42.0,Male
102954.0,Good,41.0,Female


eliminamos nulos

In [7]:
nhanes_df.isnull().sum().sort_values(ascending=False)

,0
general_health_condition,2402
age,0
gender,0


In [9]:
nhanes_df.isnull().sum().sum()

2402

In [10]:
nhanes_df.shape

(8366, 3)

eliminando nulos

In [11]:
nhanes_df_clean = (
    nhanes_df
    .dropna()
)
nhanes_df_clean.shape

(5964, 3)

# ONE HOT ENCODING CON SKLEARN

In [12]:
nhanes_df_clean['general_health_condition'].value_counts()

,count
general_health_condition,
Good,2454
Very good,1544
Fair or,1175
Excellent,619
Poor?,172


## CREAMOS EL TRANSFORMADOR

In [13]:
transformer = sklearn.compose.make_column_transformer(
    (sklearn.preprocessing.OrdinalEncoder(),["gender"]),
    (sklearn.preprocessing.OneHotEncoder(),["general_health_condition"]),
    remainder="passthrough"
)

# APLICAMOS EL TRANSFORMADOR A NUESTRO DATASET PARA LA CODIFICACIÓN

In [ ]:
nhanes_transformed_df = (
    pd.DataFrame(
        transformer.fit_transform(nhanes_df_clean),
        columns = transformer.get_feature_names_out(),
        index=nhanes_df_clean.index
    )
    .rename_columns(
        function = lambda x: x.removeprefix("ordinalencoder__")
    )
    .rename_columns(
        function = lambda x: x.removeprefix("onehotencoder__general_health_condition_")
    )
    .rename_columns(
        function = lambda x: x.removeprefix("remainder__")
    )
)
nhanes_transformed_df

/usr/local/lib/python3.11/dist-packages/pandas_flavor/register.py:157: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


,gender,Excellent,Fair or,Good,Poor?,Very good,nan,age
SEQN,,,,,,,,
93703.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
93704.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
93705.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,66.0
93706.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,18.0
93707.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...
102952.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,70.0
102953.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,42.0
102954.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0
